**（1）代码部分**

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
train_data_path = 'data-ML.csv'
test_data_path = 'data_test.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [3]:
train_data.tail()

,date,code,y,x_0,x_1,x_2,x_3,x_4,x_5,x_6,...,x_19,x_20,x_21,x_22,x_23,x_24,x_25,x_26,x_27,x_28
31156,241,124,-0.127532,-0.108338,-0.108890,-0.107348,-0.108578,-0.656338,-0.704451,-0.508945,...,-0.107590,-0.108307,-0.108331,-0.107160,-0.107703,-0.107611,-0.108556,-0.108372,-0.107686,-0.107492
31157,241,125,-0.418319,-0.205022,-0.203531,-0.204336,-0.202912,-0.748598,-0.787103,-0.701324,...,-0.203608,-0.203424,-0.204418,-0.201016,-0.202098,-0.201792,-0.202753,-0.202379,-0.202881,-0.202239
31158,241,126,0.093849,-0.220328,-0.216690,-0.220113,-0.216993,0.455691,1.186600,0.133938,...,-0.218690,-0.218992,-0.219554,-0.216605,-0.218371,-0.218316,-0.216912,-0.217480,-0.218867,-0.218569
31159,241,127,-0.540083,-0.214812,-0.211793,-0.215237,-0.213479,1.353480,1.962952,0.305835,...,-0.213855,-0.214036,-0.213563,-0.213043,-0.214655,-0.213908,-0.213250,-0.214016,-0.215259,-0.214204
31160,241,128,-1.282562,-0.059840,-0.060004,-0.058656,-0.060545,3.188366,3.758701,5.198595,...,-0.055931,-0.056681,-0.059642,-0.052446,-0.057986,-0.057476,-0.060750,-0.054918,-0.058649,-0.057281


In [4]:
train_data.shape

(31161, 32)

In [5]:
test_data.head()

,date,code,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_19,x_20,x_21,x_22,x_23,x_24,x_25,x_26,x_27,x_28
0,0,0,3.780489,3.971441,3.892044,3.995554,4.278905,3.627402,2.416898,1.895266,...,3.839883,3.850517,3.887270,3.773767,3.793691,3.826716,3.990291,3.831193,3.814994,3.835216
1,0,1,-0.159670,-0.159482,-0.159243,-0.158023,-0.763510,-0.814598,-0.831288,-0.789755,...,-0.158495,-0.159493,-0.160545,-0.157573,-0.157757,-0.158640,-0.157980,-0.157772,-0.158417,-0.158787
2,0,2,-0.045428,-0.045512,-0.042968,-0.042670,-0.549088,-0.541655,-0.624320,-0.370413,...,-0.041661,-0.045551,-0.045309,-0.046442,-0.044814,-0.048065,-0.042872,-0.045184,-0.044059,-0.047205
3,0,3,-0.175660,-0.177391,-0.179047,-0.177343,3.885955,5.151858,5.095823,5.862515,...,-0.174667,-0.175792,-0.179078,-0.180495,-0.177532,-0.185748,-0.177268,-0.179197,-0.175522,-0.184364
4,0,4,-0.094343,-0.098661,-0.102046,-0.102826,1.861440,1.621870,1.595707,1.272282,...,-0.097124,-0.085427,-0.097757,-0.096827,-0.094184,-0.082397,-0.102801,-0.098022,-0.095919,-0.083789


In [6]:
test_data.shape

(10439, 31)

In [7]:
X_train = train_data.iloc[:, 3:].replace(0, pd.NA)
X_train = X_train.fillna(X_train.mean())

y_train = train_data.iloc[:, 2].to_frame().replace(0, pd.NA)
y_train = y_train.fillna(y_train.mean())

X_test = test_data.iloc[:, 2:].replace(0, pd.NA)
X_test = X_test.fillna(X_test.mean())

In [8]:
#X_train.head()

In [9]:
#y_train.head()

In [10]:
scaler = MinMaxScaler(feature_range=(0,1)) 
 
X_train = scaler.fit_transform(X_train)  
y_train = scaler.fit_transform(y_train)
X_test = scaler.fit_transform(X_test)

#scaler = StandardScaler()  
X_train = scaler.fit_transform(X_train)  
y_train = scaler.fit_transform(y_train) 

In [11]:
#X_train

In [12]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

torch_dataset = torch.utils.data.TensorDataset(X_train, y_train)  

#这里日期被脱敏，所以我用了基本的分隔方式；只要有索引就可以根据需要特定分割。
torch.manual_seed(seed=2020) 
train, validation = torch.utils.data.random_split(torch_dataset, [11161, 20000])  

In [13]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.4)

        self._init_weights()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data.fill_(0.01)

input_dim = X_train.shape[1] 
model = MLP(input_dim)

criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.0001) 

def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            if torch.isnan(loss):
                continue
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1) 
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, train_loader, criterion, optimizer, epochs=28)

Epoch 1, Loss: 0.6932820081710815
Epoch 2, Loss: 0.528477668762207
Epoch 3, Loss: 0.13256670534610748
Epoch 4, Loss: 0.08730072528123856
Epoch 5, Loss: 0.08226903527975082
Epoch 6, Loss: 0.05253587290644646
Epoch 7, Loss: 0.06567157804965973
Epoch 8, Loss: 0.0623103529214859
Epoch 9, Loss: 0.042090319097042084
Epoch 10, Loss: 0.028348775580525398
Epoch 11, Loss: 0.023993775248527527
Epoch 12, Loss: 0.025548027828335762
Epoch 13, Loss: 0.03093358688056469
Epoch 14, Loss: 0.02971429005265236
Epoch 15, Loss: 0.026814715936779976
Epoch 16, Loss: 0.02819668874144554
Epoch 17, Loss: 0.017304575070738792
Epoch 18, Loss: 0.021278977394104004
Epoch 19, Loss: 0.017577312886714935
Epoch 20, Loss: 0.01614570803940296
Epoch 21, Loss: 0.020704416558146477
Epoch 22, Loss: 0.01979103870689869
Epoch 23, Loss: 0.01506909541785717
Epoch 24, Loss: 0.022787747904658318
Epoch 25, Loss: 0.01943368650972843
Epoch 26, Loss: 0.012533999048173428
Epoch 27, Loss: 0.016152113676071167
Epoch 28, Loss: 0.01188028231

In [14]:
model.eval()

with torch.no_grad():
    predictions = model(X_test)

y_prediction = scaler.inverse_transform(predictions)

In [15]:
y_prediction

array([[0.43436307],
       [0.46450618],
       [0.46679857],
       ...,
       [0.46455109],
       [0.46432781],
       [0.43754569]])

In [16]:
np.savetxt('y_prediction.csv', y_prediction, delimiter=',')

**（2）执行超参数搜索：**

+ 首先，常见的超参数包括学习率、批大小、网络层数、每层的神经元数量等，我们需要确定它们取值范围或候选值；

+ 其次，我们用网格搜索、随机搜索或基于梯度的方法（如Hyperband、Optuna等）遍历超参数空间；

+ 最后，对于每一组超参数，使用训练数据训练模型，并使用验证集评估模型的性能。